In [1]:
import numpy as np
from significance_of_mean_cuda import significance_of_mean_cuda
from scipy.stats import mannwhitneyu, ttest_ind
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils import qvalues, getdf



In [2]:
def exactTest(A,B, bins=10, one_side=True):
    SGM = significance_of_mean_cuda(bins, dtype_v=np.uint16,dtype_A=np.uint64)
    SGM.run(np.asarray(A),np.asarray(B), midP=True)
    if one_side:
        return SGM.get_p_values()
    else:
        return [2 * min( p, (1-p)) for p in SGM.get_p_values()]

In [3]:
def getSynteticData(func, setN=20, sampleN=2_000, mean=0, std=1):
    np.random.seed(10)
    AN, BN = [func(mean,std,setN) for i in range(sampleN)], [func(0,std,setN) for i in range(sampleN)]
    return AN, BN

In [4]:
A,B = getSynteticData(np.random.normal, 500, 10)

In [9]:
exactTest(A,B,50)

array([ 9888.45031839, 11741.68275204, 10995.24936561, 10942.01611315,
       10338.52681329, 11686.05597028, 11504.83365585, 12420.89347714,
       11380.49110725, 11525.29214672])

In [7]:
n = A[0]

In [8]:
n.nbytes

4000

In [ ]:

int64arr = np.ones((1024, 1024), dtype=np.uint64)
int16arr = np.ones((1024, 1024), dtype=np.uint16)
int16arr = np.ones((1024, 1024), dtype=np.float64)


In [ ]:
int64arr.nbytes

In [ ]:
int16arr.nbytes

In [ ]:
%load_ext cython
%load_ext rpy2.ipython



In [ ]:
%%R

require(ggplot2)
install.packages("devtools")
library("devtools")
install_github("bdsegal/fastPerm")

library(fastPerm)

In [ ]:
np.random.seed(10)

In [ ]:
%%R
    runFastPerm <- function(x,y,output, n_samples) {
        e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
      
            mStopDiffMean(x[[i]], y[[i]])
            valX = fastPerm(x[[i]], y[[i]], testStat = diffMean)
            
            valX <-unlist(valX)

      
            output[[i]] <- as.numeric(valX[1])
      
            }
    

    return(output)


    }

In [ ]:
def multiple_plot(df, save_name):
    #sns.set(font_scale=1.5)
    sns.set(style="white")
    sns.set_context("talk")
    low = min(df["Theoretical p-value"])
    hi = max(df["Theoretical p-value"])
    g=sns.lmplot(x='Theoretical p-value', y ='Observed p-value', data=df, 
                  fit_reg=False, height=7, truncate=True, scatter_kws={"s": 15}, hue="Test")
    
    g.set(xscale="log", yscale="log")
    g.set(xlabel='Theoretical $\it{p}$ value', ylabel='Observed $\it{p}$ value')
    axes = g.axes
    g.set(ylim=(low,hi), xlim=(low,hi))
    plt.plot([low,hi],[low,hi], "k", linewidth=1)
    sns.set_style("ticks")
    sns.despine()
    g.fig.tight_layout()
    g.savefig(save_name)

In [ ]:
def sns_plot(df, save_name=None):
    #sns.set(font_scale=1.5)
    sns.set(style="white")
    sns.set_context("talk")
    c1, c2 = df.columns
    low = min(min(df[c1]), min(df[c2]))
    hi = max(max(df[c1]), max(df[c2]))
    
    g=sns.lmplot(x=c1, y =c2, data=df, 
                  fit_reg=False, height=7, truncate=True, scatter_kws={"s": 15})
    
    g.set(xscale="log", yscale="log")
    axes = g.axes
    g.set(ylim=(low,hi), xlim=(low,hi))
    plt.plot([low,hi],[low,hi], "k", linewidth=1)
    sns.set_style("ticks")
    sns.despine()
    g.fig.tight_layout()
    if save_name:
        g.savefig(save_name)

In [ ]:

    
def exactTest(A,B, bins=10, one_side=True):
    SGM = significance_of_mean_cuda(bins, dtype_v=np.uint32,dtype_A=np.float64, )
    SGM.run(np.asarray(A),np.asarray(B), midP=True)
    if one_side:
        return SGM.get_p_values()
    else:
        return [2 * min( p, (1-p)) for p in SGM.get_p_values()]

def MWU(A, B, one_side=True):
    p_mw = list()
    for a,b in zip(A, B):
        if one_side:
            p_mw.append(mannwhitneyu(a,b, alternative="less")[1])
        else:
            p_mw.append(mannwhitneyu(a,b, alternative="two-sided")[1])
    return p_mw

def ttests(A,B, one_side=True):
    p_t = list()
    for x, y in zip(A, B):
        t, p = ttest_ind(y, x)
        if one_side:
            p = p/2
            if t<0:
                p = 1-p
        p_t.append(p)
    return p_t


In [ ]:
def fastPerm(A,B):
    print(A.shape)
    x = pd.DataFrame(A, columns=list(range(A.shape[1]))).T
    y = pd.DataFrame(B, columns=list(range(B.shape[1]))).T
    n_samples = x.shape[1]
    outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T
    
    %R -i x
    %R -i y
    %R -i outputDf
    %R -i n_samples
    
    
    %R out <- runFastPerm(x,y, outputDf, n_samples)
    
    
    %R -o out
    return list(out.T["output"].values)

In [ ]:
class createQvaluePlot:
    def __init__(self, pExactList=None, pMWUList=None, pTtestList=None, pFastPermList=None):
        if isinstance(pExactList, list):
            self.df_e = qvalues(self.getPvalueDf(pExactList), pi0=None)
        else:
            self.df_e = None
        
        if isinstance(pMWUList, list):
            self.df_mw = qvalues(self.getPvalueDf(pMWUList), pi0=None)
        else:
            self.df_mw = None
        
        if isinstance(pTtestList, list):
            self.df_tt = qvalues(self.getPvalueDf(pTtestList), pi0=None)
        else:
            self.df_tt = None
            
        if isinstance(pFastPermList, list):
            self.df_fp = qvalues(self.getPvalueDf(pFastPermList), pi0=None)
        else:
            self.df_fp = None
            
    
    def getPvalueDf(self, pList):
        return pd.DataFrame({"p":np.array(pList).copy()})
    
    def preparePandas(self, q_values, cumulative, test="Parallel Green"):
        preparePd = list()
        for q, cum in zip(q_values, cumulative):
            preparePd.append([q, cum, test])
        return preparePd
    
    def filerByQval(self, df, treshold=0.1):
        return df["q"][df["q"]<treshold]
    
    def getPlotData(self, df, test="Parallel Green"):
        values, base = np.histogram(df, bins=120)
        cumulative = np.cumsum(values)
        data = preparePandas(base[:-1], cumulative, test)
        return data
    
    def createPlot(self):
        D = None
        if isinstance(self.df_e, pd.core.frame.DataFrame):
            data_e = self.getPlotData(self.filerByQval(self.df_e), "Parallel Green")
            if not D:
                D = data_e
            else:
                D += data_e
        
        if isinstance(self.df_mw, pd.core.frame.DataFrame):
            data_mw = self.getPlotData(self.filerByQval(self.df_mw), "Mann–Whitney U-test")
            if not D:
                D = data_mw
            else:
                D += data_mw

        
        if isinstance(self.df_tt, pd.core.frame.DataFrame):
            data_tt = self.getPlotData(self.filerByQval(self.df_tt), "$\it{t}$-test")
            if not D:
                D = data_tt
            else:
                D += data_tt
            
            
        if isinstance(self.df_fp, pd.core.frame.DataFrame):
            data_fp = self.getPlotData(self.filerByQval(self.df_fp), "FastPerm")
            if not D:
                D = data_fp
            else:
                D += data_fp

        if D:
            pdData = pd.DataFrame(D, columns=['%','n', 'Method'])
            self.qPlot(pdData)
    
    def qPlot(self, df):
        a4_dims = (11.7, 8.27)

        fig, ax = plt.subplots(figsize=a4_dims)

        sns.set(style="white")
        sns.set_context("talk")

        snsPlot = sns.lineplot(x="%", y="n",
             hue="Method",
             data=df, ax=ax)

        plt.ylabel(r"$n_{proteins}$",fontsize=20)
        
        
        plt.xlabel(r"$q-value\ (\%)$",fontsize=15)

        sns.set_style("ticks")
        sns.despine()
        plt.tight_layout()
        plt.savefig("figures/experiment/n_proteins")

In [ ]:
def getSynteticData(func, setN=20, sampleN=2_000, mean=0, std=1):
    np.random.seed(10)
    AN, BN = [func(mean,std,setN) for i in range(sampleN)], [func(0,std,setN) for i in range(sampleN)]
    return AN, BN

In [ ]:
from scipy import stats
def r2(x, y):
    #return stats.pearsonr(x, y)[0] ** 2
    #return mean_squared_error(x,y)
    z=(x-y)/y
    z_abs = np.abs(z)
    
def my_scatter_plot(df):
    sns.set(style="white")
    sns.set_context("talk")
    low = min(df["Theoretical p-value"])
    hi = max(df["Theoretical p-value"])
    f, ax = plt.subplots(figsize=(7, 7))
    #ax.set(xscale="log", yscale="log")
    
    #err2 = r2(df["Observed p-value"], df["Theoretical p-value"])
    
    g=sns.regplot(x='Theoretical p-value', y ='Observed p-value', data=df,  ax=ax, fit_reg=False, 
                  scatter_kws={"s": 5})
    g.plot([low,hi], [low,hi], 'k-', linewidth=.5)
    
    sns.despine()
    #plt.title("Mean relative error : {}".format(np.around(err2,6)))
    f.tight_layout()


In [ ]:
def getPrediction(A,B):
    Data = dict()
    
    Data["eList"] = exactTest(B, A, 35, False)
    Data["fpList"] = fastPerm(np.array(A),np.array(B))
    Data["mwuList"] = MWU(A, B, False)
    Data["ttList"] = ttests(A, B, False)
    
    Data["eVttDf"] = pd.DataFrame(data=
                                  {'Parallel Green': Data["eList"], 'Ttest': Data["ttList"] })
    Data["mwuVttDf"] = pd.DataFrame(data={'Mann–Whitney U-test': Data["mwuList"], 'Ttest': Data["ttList"]})
    Data["eVmwuDf"] = pd.DataFrame(data={'Parallel Green': Data["eList"], 'Mann–Whitney U-test': Data["mwuList"]})

    Data["eVfpDf"] = pd.DataFrame(data={'Parallel Green': Data["eList"], 'FastPerm': Data["fpList"]})
    Data["mwuVfpDf"] = pd.DataFrame(data={'Mann–Whitney U-test': Data["mwuList"], 'FastPerm': Data["fpList"]})
    Data["ttVfpDf"] = pd.DataFrame(data={'Ttest': Data["ttList"], 'FastPerm': Data["fpList"]})
                                                      
    return Data

    

    

# $A \sim N(0,1)\ and B \sim N(0,1)$. $ |A|=20$ and $|B|=20$ with $2000$ samples.

In [ ]:
AN, BN = getSynteticData(np.random.normal,mean=0,sampleN=20_000)
%time DataNormNoDiff = getPrediction(AN, BN)

In [ ]:
fpDf = getdf(DataNormNoDiff["fpList"], 20_000, "FastPerm")
exactDf = getdf(DataNormNoDiff["eList"], 20_000, "Parallel Green")
ttDf = getdf(DataNormNoDiff["ttList"], 20_000, "$\it{t}$ test")
mwuDf = getdf(DataNormNoDiff["mwuList"], 20_000, "Mann–Whitney $\it{U}$ test")

In [ ]:
multiple_plot(pd.concat((fpDf, exactDf, ttDf, mwuDf)), "figures/calibration/largeSample_10000_20/allTests_0_1_50")
              
              

In [ ]:
sns_plot(DataNormNoDiff["eVttDf"]) 
sns_plot(DataNormNoDiff["mwuVttDf"])
sns_plot(DataNormNoDiff["eVmwuDf"])
sns_plot(DataNormNoDiff["eVfpDf"])
sns_plot(DataNormNoDiff["ttVfpDf"])
sns_plot(DataNormNoDiff["mwuVfpDf"])

In [ ]:

createQvaluePlot(DataNormNoDiff["eList"], 
                 DataNormNoDiff["mwuList"], 
                 DataNormNoDiff["ttList"],
                 DataNormNoDiff["fpList"]
                ).createPlot()



# $A \sim N(0,1)\ and B \sim N(0.5,1)$. $ |A|=20$ and $|B|=20$ with $2000$ samples.

In [ ]:
AN, BN = getSynteticData(np.random.normal, mean=0.5, sampleN=8000)
DataNormDiff = getPrediction(AN, BN)

In [ ]:
sns_plot(DataNormDiff["eVttDf"]) 
sns_plot(DataNormDiff["mwuVttDf"])
sns_plot(DataNormDiff["eVmwuDf"])
sns_plot(DataNormDiff["eVfpDf"])
sns_plot(DataNormDiff["ttVfpDf"])
sns_plot(DataNormDiff["mwuVfpDf"])

In [ ]:
createQvaluePlot(DataNormDiff["eList"], 
                 DataNormDiff["mwuList"], 
                 DataNormDiff["ttList"],
                 DataNormDiff["fpList"]
                ).createPlot()



# $A \sim logN(0,1)\ and B \sim logN(0,1)$. $ |A|=20$ and $|B|=20$ with $2000$ samples.

In [ ]:
AN, BN = getSynteticData(np.random.lognormal, mean=0, sampleN=20_000)
DataLogNormNonDiff = getPrediction(AN, BN)

In [ ]:
sns_plot(DataLogNormNonDiff["eVttDf"]) 
sns_plot(DataLogNormNonDiff["mwuVttDf"])
sns_plot(DataLogNormNonDiff["eVmwuDf"])
sns_plot(DataLogNormNonDiff["eVfpDf"])
sns_plot(DataLogNormNonDiff["ttVfpDf"])
sns_plot(DataLogNormNonDiff["mwuVfpDf"])

In [ ]:
fpDf = getdf(DataLogNormNonDiff["fpList"], 20_000, "FastPerm")
exactDf = getdf(DataLogNormNonDiff["eList"], 20_000, "Parallel Green")
ttDf = getdf(DataLogNormNonDiff["ttList"], 20_000, "$\it{t}$ test")
mwuDf = getdf(DataLogNormNonDiff["mwuList"], 20_000, "Mann–Whitney $\it{U}$ test")

In [ ]:
multiple_plot(pd.concat((fpDf, exactDf, ttDf, mwuDf)), "figures/calibration/largeSample_10000_20/allTestsLog_0_1_50")


In [ ]:
createQvaluePlot(DataLogNormNonDiff["eList"], 
                 DataLogNormNonDiff["mwuList"], 
                 DataLogNormNonDiff["ttList"],
                 DataLogNormNonDiff["fpList"]
                ).createPlot()



# $A \sim logN(0,1)\ and B \sim logN(0.5,1)$. $ |A|=20$ and $|B|=20$ with $2000$ samples.

In [ ]:
AN, BN = getSynteticData(np.random.lognormal, mean=0.5, sampleN=20_000)
DataLogNormDiff = getPrediction(AN, BN)

In [ ]:
sns_plot(DataLogNormDiff["eVttDf"]) 
sns_plot(DataLogNormDiff["mwuVttDf"])
sns_plot(DataLogNormDiff["eVmwuDf"])
sns_plot(DataLogNormDiff["eVfpDf"])
sns_plot(DataLogNormDiff["ttVfpDf"])
sns_plot(DataLogNormDiff["mwuVfpDf"])

In [ ]:
createQvaluePlot(DataLogNormDiff["eList"], 
                 DataLogNormDiff["mwuList"], 
                 DataLogNormDiff["ttList"],
                 DataLogNormDiff["fpList"]
                ).createPlot()


# Experiemntal data: There are 80 patients without triple negative (NTN) and 26 patients with triple negtive (TN), and with a total of 8051 quantified genes.

In [ ]:
NotTNP_df = pd.read_csv("data/notTNPdf", index_col=0)
TNP_df = pd.read_csv("data/TNPdf", index_col=0)
NotTNPList = [x for x in NotTNP_df.values]
TNP_dfList = [x for x in TNP_df.values]

In [ ]:
DataExperiemnt = getPrediction(NotTNPList, TNP_dfList)

In [ ]:
sns_plot(DataExperiemnt["eVttDf"]) 
sns_plot(DataExperiemnt["mwuVttDf"])
sns_plot(DataExperiemnt["eVmwuDf"])
sns_plot(DataExperiemnt["eVfpDf"])
sns_plot(DataExperiemnt["ttVfpDf"])
sns_plot(DataExperiemnt["mwuVfpDf"])

In [ ]:
createQvaluePlot(DataExperiemnt["eList"], 
                 DataExperiemnt["mwuList"], 
                 DataExperiemnt["ttList"],
                 DataExperiemnt["fpList"]
                ).createPlot()


In [ ]:
pFastPermList = fastPerm(np.array(TNP_dfList),np.array(NotTNPList))
pExactList = exactTest(TNP_dfList, NotTNPList, 30, False)
pMWUList = MWU(NotTNPList, TNP_dfList, False)
pTtestList = ttests(NotTNPList, TNP_dfList, False)

In [ ]:
dataExactTtest = pd.DataFrame(data={'parallelized': pExactList, 'ttest': pTtestList})
dataMWUTtest = pd.DataFrame(data={'MWU': pMWUList, 'ttest': pTtestList})
dataEaxactMWU = pd.DataFrame(data={'parallelized': pExactList, 'MWU': pMWUList})

dataExactFastPerm = pd.DataFrame(data={'parallelized': pExactList, 'fastPerm': pFastPermList})
dataMWUFastPerm = pd.DataFrame(data={'MWU': pMWUList, 'fastPerm': pFastPermList})
dataTtestastPerm = pd.DataFrame(data={'ttest': pTtestList, 'fastPerm': pFastPermList})


In [ ]:
sns_plot(dataExactTtest)

In [ ]:
sns_plot(dataEaxactMWU)

In [ ]:
sns_plot(dataMWUTtest)


In [ ]:
sns_plot(dataTtestastPerm)

In [ ]:
sns_plot(dataMWUFastPerm)

In [ ]:
type(dataTtestastPerm)

In [ ]:
df_e = pd.DataFrame({"p":np.array(pExactList).copy()})
df_mw = pd.DataFrame({"p":np.array(pMWUList).copy()})
df_tt = pd.DataFrame({"p":np.array(pTtestList).copy()})
df_fp = pd.DataFrame({"p":np.array(pFastPermList).copy() })

df_e = qvalues(df_e, pi0=None)
df_mw = qvalues(df_mw, pi0=None)
df_tt = qvalues(df_tt, pi0=None)
df_fp = qvalues(df_fp, pi0=None)

In [ ]:
data_e = df_e["q"][df_e["q"]<0.1]
data_mw = df_mw["q"][df_mw["q"]<0.1]
data_tt = df_tt["q"][df_tt["q"]<0.1]
data_fp = df_fp["q"][df_fp["q"]<0.1]


In [ ]:
def preparePandas(q_values, cumulative, test="parallelized shift"):
    preparePd = list()
    for q, cum in zip(q_values, cumulative):
        preparePd.append([q, cum, test])
    return preparePd

In [ ]:
values, base = np.histogram(data_e, bins=120)
cumulative = np.cumsum(values)
exactData = preparePandas(base[:-1], cumulative)

values, base = np.histogram(data_mw, bins=120)
cumulative = np.cumsum(values)
mwuData = preparePandas(base[:-1], cumulative, "MWU")

values, base = np.histogram(data_tt, bins=120)
cumulative = np.cumsum(values)
ttData = preparePandas(base[:-1], cumulative, "t test")

values, base = np.histogram(data_fp, bins=120)
cumulative = np.cumsum(values)
fpData = preparePandas(base[:-1], cumulative, "fastPerm")

In [ ]:
D = exactData +  mwuData + ttData + fpData
pdData = pd.DataFrame(D, columns=['%','n', 'Method'])

In [ ]:
a4_dims = (11.7, 8.27)

fig, ax = plt.subplots(figsize=a4_dims)

sns.set(style="white")
sns.set_context("talk")

snsPlot = sns.lineplot(x="%", y="n",
             hue="Method",
             data=pdData, ax=ax)

plt.ylabel(r"$n_{proteins}$",fontsize=20)
        
        
plt.xlabel(r"$q-value\ (\%)$",fontsize=15)

sns.set_style("ticks")
sns.despine()
plt.tight_layout()
plt.savefig("figures/experiment/n_proteins")


In [ ]:
np.random.seed(10)
set_n = 20
sample_n = 2_000
AN, BN = [np.random.normal(0.5,1,set_n) for i in range(sample_n)], [np.random.normal(0,1,set_n) for i in range(sample_n)]

In [ ]:
pFastPermListNormDiff = fastPerm(np.array(AN),np.array(BN))
pExactListNormDiff = exactTest(BN, AN, 50, False)
pMWUListNormDiff = MWU(AN, BN, False)
pTtestListNormDiff = ttests(AN, BN, False)

In [ ]:
dataExactTtestNormDiff = pd.DataFrame(data={'parallelized': pExactListNormDiff, 'ttest': pTtestListNormDiff})
dataMWUTtestNormDiff = pd.DataFrame(data={'MWU': pMWUListNormDiff, 'ttest': pTtestListNormDiff})
dataEaxactMWUNormDiff = pd.DataFrame(data={'parallelized': pExactListNormDiff, 'MWU': pMWUListNormDiff})

dataExactFastPermNormDiff = pd.DataFrame(data={'parallelized': pExactListNormDiff, 'fastPerm': pFastPermListNormDiff})
dataMWUFastPermNormDiff = pd.DataFrame(data={'MWU': pMWUListNormDiff, 'fastPerm': pFastPermListNormDiff})
dataTtestastPermNormDiff = pd.DataFrame(data={'ttest': pTtestListNormDiff, 'fastPerm': pFastPermListNormDiff})


In [ ]:
sns_plot(dataExactTtestNormDiff)

In [ ]:
sns_plot(dataMWUTtestNormDiff)

In [ ]:
sns_plot(dataEaxactMWUNormDiff)

In [ ]:
sns_plot(dataExactFastPermNormDiff)

In [ ]:
sns_plot(dataMWUFastPermNormDiff)

In [ ]:
sns_plot(dataTtestastPermNormDiff)

In [ ]:
Q = createQvaluePlot(pExactListNormDiff, pMWUListNormDiff, pTtestList, pFastPermListNormDiff).createPlot()
Q.createPlot()

In [ ]:
set_n = 20
sample_n = 2_000
AN, BN = [np.random.lognormal(0,2,set_n) for i in range(sample_n)], [np.random.lognormal(0,2,set_n) for i in range(sample_n)]


pFastPermListLogDiff = fastPerm(np.array(AN),np.array(BN))
pExactListLogDiff = exactTest(BN, AN, 50, False)
pMWUListLogDiff = MWU(AN, BN, False)
pTtestListLogDiff = ttests(AN, BN, False)

In [ ]:
keep_ix = np.array(pExactListLogDiff) > 10**-10

In [ ]:
dataExactTtestLogDiff = pd.DataFrame(data={'parallelized': np.array(pExactListLogDiff)[keep_ix], 'ttest': np.array(pTtestListLogDiff)[keep_ix]})
dataMWUTtestLogDiff = pd.DataFrame(data={'MWU': np.array(pMWUListLogDiff)[keep_ix], 'ttest': np.array(pTtestListLogDiff)[keep_ix]})
dataEaxactMWULogDiff = pd.DataFrame(data={'parallelized': np.array(pExactListLogDiff)[keep_ix], 'MWU': np.array(pMWUListLogDiff)[keep_ix]})

dataExactFastPermLogDiff = pd.DataFrame(data={'parallelized': np.array(pExactListLogDiff)[keep_ix], 'fastPerm': np.array(pFastPermListLogDiff)[keep_ix]})
dataMWUFastPermLogDiff = pd.DataFrame(data={'MWU': np.array(pMWUListLogDiff)[keep_ix], 'fastPerm': np.array(pFastPermListLogDiff)[keep_ix]})
dataTtestastPermLogDiff = pd.DataFrame(data={'ttest': np.array(pTtestListLogDiff)[keep_ix], 'fastPerm': np.array(pFastPermListLogDiff)[keep_ix]})


In [ ]:
sns_plot(dataExactTtestLogDiff)

In [ ]:
sns_plot(dataMWUTtestLogDiff)

In [ ]:
sns_plot(dataEaxactMWULogDiff)

In [ ]:
sns_plot(dataExactFastPermLogDiff)

In [ ]:
sns_plot(dataMWUFastPermLogDiff)

In [ ]:
sns_plot(dataTtestastPermLogDiff)

In [ ]:
list(np.array(pFastPermListLogDiff)[keep_ix])

In [ ]:
Q = createQvaluePlot(
                     list(np.array(pMWUListLogDiff)[keep_ix]),
                     list(np.array(pTtestListLogDiff)[keep_ix]))
Q.createPlot()